In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import numpy as np
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import calendar

In [3]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

def Extract_MTA_Data(first_date,last_date):
    #Taking 1 day extra on both sides to handle boundary values
    first_date_x = first_date - pd.Timedelta(days=1)        
    last_date_x = last_date + pd.Timedelta(days=1)
    
    #Creating URL to import data and the format for each week of data
    url_format = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"    
    startweek = first_date_x - dt.timedelta(((first_date_x.weekday() + 1) % 7)+1)
    currentweek = startweek
    lastweek = last_date_x + pd.Timedelta(days=7)
    weekly = []
    
    #Importing raw data from MTA
    while currentweek <= lastweek:
        dateformat = str(currentweek.year)[2:4] + str(currentweek.month).zfill(2) + str(currentweek.day).zfill(2)
        url = url_format.format(dateformat)
        temp = pd.read_csv(url, parse_dates=['DATE'], keep_date_col=True)
        temp = temp.rename({'EXITS                                                               ':'EXITS'},axis=1)
        weekly.append(temp)
        currentweek += pd.Timedelta(days=7)
    
    #Concatenating all weeks together
    temp_df = pd.concat(weekly, axis=0, ignore_index=True).reset_index(drop=True)
    temp_df = Get_Real_Turnstile_Count(temp_df)
    temp_df = Process_MTA_Data(temp_df,bins=6)
    
    #Keeping only data within bounds
    return temp_df[(temp_df['DATE'] >= first_date) & (temp_df['DATE'] <= last_date)].reset_index(drop=True)
  
def Get_Real_Turnstile_Count(df):
    x_=[]
    df = df.groupby(['STATION','C/A','UNIT','SCP','DATE','TIME','DESC']).agg({"EXITS":'mean',"ENTRIES":'mean'})
    df = df.reset_index(drop=False).set_index(['STATION','C/A','UNIT','SCP','DESC'])
    for n,i in enumerate(df.index.unique()):
        temp = df[df.index == i]
        temp['EXIT'] = temp['EXITS']-temp['EXITS'].shift()
        temp['ENTRY'] = temp['ENTRIES']-temp['ENTRIES'].shift()
        temp = temp.reset_index(drop=True)
        x_.append(temp)
    df = pd.concat(x_, axis=0, ignore_index=True)
    
    #Handling Bad data
    df.loc[df['EXIT']<0,'EXIT']=df.loc[df['EXIT']<0,'EXITS']
    df.loc[df['ENTRY']<0,'ENTRY']=df.loc[df['ENTRY']<0,'ENTRIES']
    df.loc[df['EXIT']>np.nanpercentile(df['EXIT'],99),'EXIT']=np.nanpercentile(df['EXIT'],50)
    df.loc[df['ENTRY']>np.nanpercentile(df['ENTRY'],99),'ENTRY']=np.nanpercentile(df['ENTRY'],50)
    df = df[['DATE','TIME','EXIT','ENTRY']]
    return df

def Process_MTA_Data(df,bins=6):
    df['Hour'] = df['TIME'].str[:2].apply(int)
    df['HOUR_BIN'] = df['Hour'].apply(binning)
    df = df.drop('Hour',axis=1)
    df = df.groupby(['DATE','HOUR_BIN']).sum().reset_index(drop=False)
    return df

def binning(x,bins=6):
    return x-(x%(24//bins))

def monthfilter(year,month):
    _, num_days = calendar.monthrange(year,month)
    first_day = pd.Timestamp('{}-{}-{} 00:00:00'.format(year, month,1 ))
    last_day = pd.Timestamp('{}-{}-{} 00:00:00'.format(year, month,num_days ))
    return first_day,last_day

In [4]:
%%time

Years = [2018,2019]
Months = range(1,13)
n=0
for Year in Years:
    for Month in Months:
        F = Extract_MTA_Data(monthfilter(Year,Month)[0],monthfilter(Year,Month)[1])
        F.to_csv('MTA_'+str(Year)+'_'+str(Month))
        n+=1
        print(n,"/",len(Years)*len(Months))

1 / 24
2 / 24
3 / 24
4 / 24
5 / 24
6 / 24
7 / 24
8 / 24
9 / 24
10 / 24
11 / 24
12 / 24
13 / 24
14 / 24
15 / 24
16 / 24
17 / 24
18 / 24
19 / 24
20 / 24
21 / 24
22 / 24
23 / 24
24 / 24
CPU times: user 3h 2min 2s, sys: 2min 45s, total: 3h 4min 47s
Wall time: 3h 7min 36s


In [22]:
import os
x_ = []
for i in os.listdir():
    if "MTA_20" in i:
        x_.append(pd.read_csv(i,parse_dates=['DATE'])[['DATE','HOUR_BIN','EXIT','ENTRY']])
DF_MTA = pd.concat(x_,axis=0)

In [32]:
DF_MTA.head()

,DATE,HOUR_BIN,EXIT,ENTRY
0,2018-01-01,0,283254.0,299794.0
1,2018-01-01,4,137862.0,146343.0
2,2018-01-01,8,151154.0,196377.0
3,2018-01-01,12,328108.0,415589.0
4,2018-01-01,16,492717.0,594243.0


In [33]:
DF_MTA.tail()

,DATE,HOUR_BIN,EXIT,ENTRY
181,2019-12-31,4,137058.0,175073.0
182,2019-12-31,8,588964.0,736485.0
183,2019-12-31,12,796791.0,997440.0
184,2019-12-31,16,973974.0,1236863.0
185,2019-12-31,20,716006.0,800870.0


In [34]:
DF_MTA.shape

(4380, 4)

In [35]:
DF_MTA = DF_MTA.sort_values(['DATE','HOUR_BIN'])

In [36]:
DF_MTA.to_csv('DF_MTA')